# Databricks SQL- Plotly, Dash

# Table Of Contents
[1. Introduction](#1.Introduction)\
[2. Why Plotly Resampler?](#2.Why_Plotly_Resampler? )\
[3. Prerequisites](#3.-Prerequisites)\
[4. High-Level Steps](#4.-High_Level_Steps)\
[5. High-Level Procedure](#4.-High_Level_Procedure)\
[6. Installation of Databricks SQL Connector for Python](#4.-Installation_of_Databrick_SQL_Connector_for_Python)\
[7. Having No time-series dataset](#7.-Having_No_time-series_dataset)\
[8. Query to SQL Wearhouse](#8.-Query_to_SQL_Wearhouse)\
[9. Plotly Resampler](#9.-Plotly_Resampler)\
[10. Raising the Stakes with Polars](#10.-Raising_the_Stakes_with_Polars)\
[11. Polars, a substitute for Pandas](#11.-Polars_a_substitute_for_Pandas)\
[12. The Datashader Alternative](#12.-The_Datashader_Alternative)\
[13. Conclusion](#13.-Conclusion)

### 1. Introduction 

This article aims to provide a thorough overview of our journey in visualizing vast time-series data stored in Databricks by utilizing advanced `aggregation and downsampling` techniques from the Plotly Resampler. Following this, we will demonstrate how these technologies are combined to create a production data application powered by Dash, optimized for `Polars`, and utilizing Plotly.

This approach allows for a more in-depth exploration of underlying patterns and trends and provides a `scalable solution` for handling and interpreting data on an industrial level. Throughout this process, we will address the challenges associated with working with large time-series datasets and provide an overview of the requirements for querying and visualizing 10¹⁰ individual data points at a time.


### 2. Why Plotly Resampler?
The Plotly Resampler is an extension of Plotly’s Python bindings, improving the scalability of line charts within an interactive toolkit by consolidating the underlying data based on the current graph view. This can significantly enhance your visualization of extensive sets of sequential data.

- Performance improvement for extensive datasets: The Plotly chart interactivity is optimized for datasets with 100,000 data points or less by default. However, for large-scale time-series data, the Plotly Resampler dynamically aggregates data based on the current scale, resulting in smoother interactions and faster load times.
- Interactive data manipulation: The tool uses callbacks to update and aggregate data as users interact with the plot, making exploring large datasets more intuitive and responsive, especially when zooming or panning.
- Flexible aggregation methods: It provides interfaces for various sequence aggregation techniques, allowing users to select or develop an algorithm that best suits their data visualization requirements.
- Enhanced user interface: The Resampler modifies the double-click behavior within a line-chart area to initiate an "Autoscale" event, improving the overall user experience.


### 3. Prerequisites
This article is intended for individuals with substantial amounts of time-series data stored in either ADLS2 Blob Storage or AWS S3 Buckets, seeking visualization and exploration methods for their data. Suppose you do not have your data. In that case, you can utilize the Raspberry Pi Azure IoT Online Simulator to stream simulated data into Blob Storage (as detailed in Article #2) or use the provided CSV in the repository to upload to Databricks and write to Delta tables using the accompanying SQL or PySpark module.



### 4. High-Level Steps
Performance improvement for extensive datasets: Typically, Plotly chart interactivity is most effective when dealing with datasets of 100,000 data points or less. When working with large-scale time-series data and aiming to interactively engage with all the data, the Plotly Resampler dynamically consolidates data based on the current scale, ensuring smoother interactions. Apart from a Delta table containing a time-series dataset, the following are required:

1. A Databricks workspace with Databricks SQL enabled (DB SQL is enabled by default in Premium Workspaces or above)
2. A DB SQL endpoint or Databricks cluster with 9.1 LTS or higher (data engineering cluster)
3. A personal access token in Databricks to authenticate SQL Endpoint via API
4. A Python development environment (>=v 3.10). It is recommended to use VSCode for a local IDE and utilize Conda or virtual env to manage dependencies, along with `black` for automatic code formatting.

### 5. High-Level Procedure
For the context of this document, it is assumed that the reader already possesses time-series data stored in the cloud or has utilized one of the data generation techniques outlined in the prerequisite section to generate a dataset. The subsequent steps undertaken were as follows:

1. Employ the Medallion Architecture to establish a pipeline that acquires raw IoT data from the cloud/source and writes to three stages of Delta
2. Utilize the Databricks SQL Connector for Python to query the Delta table on the SQL Warehouse
3. Channel data from SQL Warehouse into Arrow files (opt for Parquet if replication is desired) residing on the persistent file storage system provided with Dash Enterprise (alternatively, they can be stored in the root directory of the project folder at the expense of memory)
4. Import data from Arrow files into a Polars-optimized version of the Plotly Resampler for visualization (or import parquet files into the standard Plotly Resample)Performance improvement for extensive datasets: Typically, Plotly chart interactivity is most effective when dealing with datasets of 100,000 data points or less. When working with large-scale time-series data and aiming to interactively engage with all the data, the Plotly Resampler dynamically consolidates data based on the current scale, ensuring smoother interactions. Apart from a Delta table containing a time-series dataset, the following are required:

1. A Databricks workspace with Databricks SQL enabled (DB SQL is enabled by default in Premium Workspaces or above)
2. A DB SQL endpoint or Databricks cluster with 9.1 LTS or higher (data engineering cluster)
3. A personal access token in Databricks to authenticate SQL Endpoint via API
4. A Python development environment (>=v 3.10). It is recommended to use VSCode for a local IDE and utilize Conda or virtual env to manage dependencies, along with black for automatic code formatting.


### 6.Installation of Databricks SQL Connector for Python

!pip install databricks-sql-connector

### 7. Having No time-series dataset
If you do not possess a substantial time-series dataset for visualization purposes, this section is designed specifically for you. In case you have already stored your measurement data in Delta tables, feel free to proceed directly to Step 2: Query SQL Warehouse. 

In the context of an optimal industrial IoT pipeline, it is recommended to implement the Medallion Architecture for preprocessing data as illustrated in the diagram.



It is advisable to consider the CSV produced by the Python script as a valuable layer when referring to this article. After uploading the CSV to your Databricks File System, you have the option to transfer its data to a Delta table using SQL.

In [ ]:
#### Sample code using PySpark

%python
# File location and type
file_location = "/FileStore/auto_iot_sensor_data.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

permanent_table_name = "main.resamplerdata.auto_iot_data_bronze_sensors"
df.write.format("delta").saveAsTable(permanent_table_name)

### 8. Query to SQL Wearhouse

Once the pipeline construction is completed and data starts flowing into the Delta tables, the subsequent action involves utilizing the Databricks SQL Connector for Python to retrieve data from the SQL Warehouse and save the query results as a Parquet file in our Dash application. The provided code outlines the process of establishing a connection engine through SQLAlchemy using your Databricks credentials.

Once the pipeline construction is completed and data starts to be inserted into the Delta tables, the subsequent task involves utilizing the Databricks SQL Connector for Python to execute a query on the SQL Warehouse and retrieve the query results as a Parquet file within our Dash application. The provided code outlines the process of establishing a connection engine using SQLAlchemy and your Databricks credentials. If you are working through the GitHub repository, simply include your Databricks credentials in the .env file located in the project folder.



#### Installation 
!pip install sqlalchemy

1. A demonstration of how to extract information from Databricks using the SQLAlchemy engine and save it into Parquet files.
2. An example of extracting data from Databricks usingthe  SQLAlchemy engine and then storing it in Parquet files.

In [ ]:
from sqlalchemy import create_engine
import pandas as pd

engine_url = f"databricks://token:{token}@{host}/?http_path={path}&catalog=main&schema=information_schema"
engine = create_engine(engine_url)

# # Fetch data from the database
engine_temp_stmt = "SELECT Timestamp, EngineTemperature_C FROM main.resamplerdata.auto_iot_bronze_sensors ORDER BY Timestamp ASC LIMIT 10000000;"
oil_pressure_stmt = "SELECT Timestamp, OilPressure_psi FROM main.resamplerdata.auto_iot_bronze_sensors ORDER BY Timestamp ASC LIMIT 10000000;"
tire_pressure_stmt = "SELECT Timestamp, TirePressure_psi FROM main.resamplerdata.auto_iot_bronze_sensors ORDER BY Timestamp ASC LIMIT 10000000;"
battery_voltage_stmt = "SELECT Timestamp, BatteryVoltage_V FROM main.resamplerdata.auto_iot_bronze_sensors ORDER BY Timestamp ASC LIMIT 10000000;"


engine_temp_df = pd.read_sql_query(engine_temp_stmt, engine)
oil_pressure_df = pd.read_sql_query(oil_pressure_stmt, engine)
speed_df = pd.read_sql_query(speed_stmt, engine)
tire_pressure_df = pd.read_sql_query(tire_pressure_stmt, engine)
battery_voltage_df = pd.read_sql_query(battery_voltage_stmt, engine)

engine_temp_df.to_parquet(f"backend-data/engine_temp_df.parquet")
oil_pressure_df.to_parquet(f"backend-data/oil_pressure_df.parquet")
speed_df.to_parquet(f"backend-data/speed_df.parquet")
tire_pressure_df.to_parquet(f"backend-data/tire_pressure_df.parquet")
battery_voltage_df.to_parquet(f"backend-data/battery_voltage_df.parquet")

The information is saved in arrow files within a persistent file storage system provided by Dash Enterprise. The example above illustrates the process of saving data from Databricks into Parquet files located in a folder named back-end data within our project. Here, the data is partitioned and stored based on columns. By caching the queried data in this manner, unnecessary data transfers over the network are avoided. Additionally, the application can be configured to only process new data at scheduled intervals, reducing bandwidth usage, improving performance, and optimizing resource utilization.

### 9.Plotly Resampler
Once the data has been retrieved and stored, we can proceed with loading our Arrow files into the Resampler for visualization. Following the data extraction from Databricks, it is saved as Arrow files in our persistent file system. The Resampler utilizes Dash callbacks to register inputs and dynamically update the figure, as illustrated in the code below.



#### Callbacks demonstrating Resampler logic

In [ ]:
# --------- graph construction logic + callback ---------
@app.callback(
    [
        Output("coarse-graph", "figure"),
        Output("plotly-resampler-graph", "figure"),
        ServersideOutput("store", "data"),
    ],
    [Input("plot-button", "n_clicks"), *get_selector_states(len(name_folder_list))],
    prevent_initial_call=True,
)
def construct_plot_graph(n_clicks, *folder_list):
    it = iter(folder_list)
    file_list: List[Path] = []
    for folder, files in zip(it, it):
        if not all((folder, files)):
            continue
        else:
            files = [files] if not isinstance(files, list) else file_list
            for file in files:
                file_list.append((Path(folder).joinpath(file)))

    ctx = callback_context
    if len(ctx.triggered) and "plot-button" in ctx.triggered[0]["prop_id"]:
        if len(file_list):
            # Create two graphs, a dynamic plotly-resampler graph and a coarse graph
            dynamic_fig: FigureResampler = visualize_multiple_files(file_list)
            coarse_fig: go.Figure = go.Figure(
                FigureResampler(dynamic_fig, default_n_shown_samples=3_000)
            )

            coarse_fig.update_layout(title="<b>coarse view</b>", height=250)
            coarse_fig.update_layout(margin=dict(l=0, r=0, b=0, t=40, pad=10))
            coarse_fig.update_layout(showlegend=False)
            coarse_fig._config = coarse_fig._config.update(
                {"modeBarButtonsToAdd": ["drawrect"]}
            )

            dynamic_fig._global_n_shown_samples = 1000
            dynamic_fig.update_layout(title="<b>dynamic view<b>", height=450)
            dynamic_fig.update_layout(margin=dict(l=0, r=0, b=40, t=40, pad=10))
            dynamic_fig.update_layout(
                legend=dict(
                    orientation="h", y=-0.11, xanchor="right", x=1, font_size=18
                )
            )

            return coarse_fig, dynamic_fig, dynamic_fig
    else:
        return no_update

There are various methods to utilize and further personalize the Plotly Resampler. In this demonstration, we are showcasing a page that displays a simultaneous coarse and fine-grained view, a graph that supports multiple overlaying y-axes, and a page that allows users to add traces for comparison. A significant feature of the Plotly Resampler is its capability to update the graph based on user input. For instance, modifications to the coarse graph will automatically trigger updates to the fine-grained graph, thereby adjusting the view accordingly. Moreover, in this article, we are utilizing a Polars dataframe with the Plotly Resampler to enhance performance compared to pandas. Stay tuned for a future article that will showcase zero MB visualizations using Polars and Plotly Resampler!


Similar to all Dash charts, we can fully personalize the colors, themes, line sizes, and legend visibility to enhance readability and aesthetics. By utilizing the Resampler, we can efficiently navigate through a dataset encompassing 6 months of time-series data, handling up to 50 million rows (or more based on memory capacity), and zoom into the sub-second level with minimal latency. Additionally, it supports the integration of multiple y-axes, encouraging a comprehensive cross-analysis of insights.


The Plotly Resampler performs exceptionally well for datasets containing around 150 million to 200 million points. However, when dealing with larger datasets, memory constraints become an issue. This is where Polars comes in. By using Polars dataframes instead of Pandas inside the Resampler library, we were able to significantly reduce the resources consumed, allowing us to visualize datasets with as many as 1 billion points. Polars achieves this through lazy evaluation, Arrow memory layout, and built-in multi-threading, making it a more efficient alternative to Pandas, especially for large datasets. These performance enhancements are critical for the Plotly Resampler, enabling users to process and resample large datasets quickly and efficiently without the need for additional hardware or complex optimizations.




### 10. Raising the Stakes with Polars
Transitioning to Polars within the Resampler library has not only enabled the handling of larger datasets but has also improved processing speed for current ones. Users are now able to enjoy faster performance without sacrificing scale. The Polars Plotly Resampler library is a continuously evolving project at Plotly, with additional improvements on the horizon.



### 11.  Polars, a substitute for Pandas
Polars, a substitute for Pandas, is specifically designed for high-performance, multi-threaded data processing. There are several reasons why Polars surpasses Pandas, particularly for extensive datasets:

1. Lazy evaluation: Polarutilizees lazy evaluation, meaning computations are not immediately executed when an operation is called. Instead, they are queued up and executed in a single pass. This optimizes the sequence of operations and reduces unnecessary intermediate calculations.

2. Arrow memory layout: Polars utilizes Apache Arrow for its memory layout. Arrow provides columnar memory, ensuring that data is stored contiguously in memory, resulting in faster processing times.

3. Built-in multi-threading: While Pandas may require manual parallelization for optimal performance, Polars automatically leverages multi-threading, maximizing CPU utilization for data operations.

In the context of the Plotly Resampler, these performance improvements are crucial. When working with large datasets, the ability to efficiently process and resample data is essential. With the memory efficiencies of Polars, users can now handle datasets that were previously inaccessible without the need for additional hardware or complex optimization.


 Utilizing Polars within the Resampler library not only enabled the handling of larger datasets but also improved the processing speed for current ones. Users are no longer required to sacrifice speed or scalability.

### 12. The Datashader Alternative
Datashader’s functionality is further enhanced by its seamless integration with the revered visualization library, Bokeh, allowing for interactive plots that can captivate viewers. In the realm of geospatial data visualization, where large datasets are common and downsampling of 2D data representations is necessary, Datashader proves to be a reliable tool. However, it does have its drawbacks. For beginners, especially those unfamiliar with Python scripting, navigating Datashader can be challenging, while the Plotly Resampler can be used as a wrapper for a Plotly figure. Additionally, when used on its own, Datashader tends to offer less interactivity compared to other tools like Plotly. These limitations, along with the potential loss of finer data details in rasterized outputs, highlight areas where Datashader can be improved. (Sidenote: We are exploring ways to integrate Datashader with Plotly charts / Dash to simplify the process and make this capability more accessible to our enterprise audience).

### Conclusion
While working with large-scale time-series data visualization, the Plotly Resampler's functionalities truly work out, especially when paired with Databricks ecosystems. It's not just the capacity to manage extensive datasets that sets it apart, but also its distinctive interactive features. These, combined with extensive customization choices and significantly improved performance, establish it as an essential tool for individuals working with expansive time-series data.

Although Datashader is praiseworthy, particularly for its innovative rasterization methods and integration with Bokeh, it may not offer the same level of user-friendly interactivity as the Plotly Resampler. This distinction makes the latter the preferred option for industrial IoT time-series scenarios.

Moreover, the incorporation of Polars into the Resampler's workflow enhances the rapid advancement and flexibility of the data science toolkit. 

In conclusion, the combined capabilities of Plotly's charting library, Dash, Dash Enterprise, the Plotly Resampler, and Databricks provide an unmatched toolkit, enabling professionals to delve deeper into vast datasets and redefine the boundaries of data analytics.